In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark import Session
from snowflake.snowpark.types import StringType

session = get_active_session()


TO CONVERT merge_revenue_ifrs_dd_seized.py CODE

In [ ]:
from snowflake.snowpark import Session, DataFrame

def process_data(session: Session, event_date: str):
    source_table = "tsel_poc.raw.ifrs_bonus_seizure_poc_tokenized"
    target_table = "telkomsel_poc.curated.merge_revenue_ifrs_dd_poc_tokenized"  # Replace with your actual target table

    # Step 1: Delete existing data for the event_date AND source = 'SEIZED'
    delete_sql = f""" DELETE FROM {target_table} WHERE event_date = '{event_date}' AND source = 'SEIZED' """

    session.sql(delete_sql).collect()

    # Step 2: Build the DataFrame with new data to append
    query = f"""
     SELECT 
            trx_date,
            CONCAT(SUBSTR(timestamp_ifrs, 1, 4), '-', SUBSTR(timestamp_ifrs, 5, 2), '-', SUBSTR(timestamp_ifrs, 7, 2)) AS purchase_date,
            transaction_id,
            subs_id,
            msisdn,
            price_plan_id,
            brand,
            pre_post_flag,
            cust_type_desc,
            cust_subtype_desc,
            customer_sub_segment,
            lac,
            ci,
            lacci_id_ifrs AS lacci_id,
            node,
            CASE WHEN area_sales IS NULL OR area_sales = '' THEN 'UNKNOWN' ELSE area_sales END AS area_sales,
            CASE WHEN region_sales IS NULL OR region_sales = '' THEN 'UNKNOWN' ELSE region_sales END AS region_sales,
            CASE WHEN branch IS NULL OR branch = '' THEN 'UNKNOWN' ELSE branch END AS branch,
            CASE WHEN sub_branch IS NULL OR sub_branch = '' THEN 'UNKNOWN' ELSE sub_branch END AS subbranch,
            CASE WHEN cluster_sales IS NULL OR cluster_sales = '' THEN 'UNKNOWN' ELSE cluster_sales END AS cluster_sales,
            CASE WHEN propinsi IS NULL OR propinsi = '' THEN 'UNKNOWN' ELSE propinsi END AS provinsi,
            CASE WHEN kabupaten IS NULL OR kabupaten = '' THEN 'UNKNOWN' ELSE kabupaten END AS kabupaten,
            CASE WHEN kecamatan IS NULL OR kecamatan = '' THEN 'UNKNOWN' ELSE kecamatan END AS kecamatan,
            CASE WHEN kelurahan IS NULL OR kelurahan = '' THEN 'UNKNOWN' ELSE kelurahan END AS kelurahan,
            lacci_closing_flag,
            sigma_business_id,
            sigma_rules_id,
            sku,
            l1_payu,
            l2_service_type,
            l3_allowance_type,
            l4_product_category,
            l5_product,
            '' AS l1_ias,
            '' AS l2_ias,
            '' AS l3_ias,
            commercial_name,
            channel,
            validity AS pack_validity,
            CAST(SUM(0) AS DECIMAL(38, 15)) AS rev_per_usage,
            CAST(SUM(rev_seized) AS DECIMAL(38, 15)) AS rev_seized,
            CAST(SUM(0) AS INT) AS dur,
            CAST(SUM(remaining_quota) AS INT) AS trx,
            CAST(SUM(0) AS BIGINT) AS vol,
            cust_id,
            '' AS profile_name,
            item_id AS quota_name,
            '' AS service_filter,
            '' AS price_plan_name,
            channel_id,
            '' AS site_id,
            '' AS site_name,
            region_hlr,
            city_hlr,
            current_date() AS load_date,
            event_Date,
            'SEIZED' AS SOURCE
        FROM {source_table}
        WHERE event_date = '{event_date}'
          AND rev_seized < 100000000000
          AND pre_post_flag = '1'
        GROUP BY 
            trx_date, 
            CONCAT(SUBSTR(timestamp_ifrs, 1, 4), '-', SUBSTR(timestamp_ifrs, 5, 2), '-', SUBSTR(timestamp_ifrs, 7, 2)),
            transaction_id, subs_id, msisdn, price_plan_id, brand, pre_post_flag, cust_type_desc,
            cust_subtype_desc, customer_sub_segment, lac, ci, lacci_id_ifrs, node,
            area_sales, region_sales, branch, sub_branch, cluster_sales, propinsi,
            kabupaten, kecamatan, kelurahan, lacci_closing_flag, sigma_business_id,
            sigma_rules_id, sku, l1_payu, l2_service_type, l3_allowance_type, 
            l4_product_category, l5_product, commercial_name, channel, validity, 
            cust_id, item_id, channel_id, region_hlr, city_hlr, event_Date
    """

    df = session.sql(query)

    # Step 3: Append the query result to the target table
    df.write.mode("append").save_as_table(target_table)

    return f"Deleted and appended data for event_date: {event_date} and source = 'SEIZED'"


In [ ]:
session.sproc.register(
    func=process_data,
    name="process_data_proc_merge_revenue_ifrs_dd_seized",
    is_permanent=True,
    stage_location="@TSELPOC_STAGE",           # Replace with your actual stage
    replace=True,
    input_types=[StringType()],          # Instead of [str]
    return_type=StringType(),            # Instead of str or "STRING"
    packages=["snowflake-snowpark-python"]
)


In [ ]:
call TELKOMSEL_POC.CURATED.PROCESS_DATA_PROC_MERGE_REVENUE_IFRS_DD_SEIZED('2025-04-01')

In [ ]:
select count(*) from telkomsel_poc.curated.merge_revenue_ifrs_dd_poc_tokenized;